<a href="https://colab.research.google.com/github/jisu9414/Data-structure/blob/main/(%EC%B5%9C%EC%A2%85)calculate_PPV_TPR(by%20chr%2C%20position%2C%20ref%2C%20alt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd

In [15]:
df1 = pd.read_table(input(""), sep='\t')
df2 = pd.read_table(input(""), sep='\t')

/content/fusion_amp_unfiltered.snv.annovar.hg19_multianno.txt
/content/answer_amp.txt


In [16]:
df1_pred = df1.iloc[:, [0, 1, 3, 4]]
df2_ans = df2.iloc[:, [0, 1, 2, 3]]

In [17]:
df1_pred

,Chr,Start,Ref,Alt
0,1,3745834,A,G
1,1,3745836,T,C
2,1,3745846,A,G
3,1,3745848,T,C
4,1,3745849,T,C
...,...,...,...,...
9823,22,50897995,G,A
9824,22,50898007,C,T
9825,22,50898026,G,A
9826,22,50898042,C,T


In [18]:
df2_ans

,Chrom,Position,Ref,Alt
0,chr1,3745924,G,A
1,chr1,10523666,G,A
2,chr1,11024271,G,T
3,chr1,15708540,A,G
4,chr1,17296361,C,G
...,...,...,...,...
535,chr22,41605809,G,C
536,chr22,42076330,G,A
537,chr22,43459864,G,A
538,chr22,45828202,C,T


In [20]:
df2_ans["Chrom"] = df2_ans["Chrom"].str.replace(pat=r'chr', repl=r'')
df2_ans

,Chrom,Position,Ref,Alt
0,1,3745924,G,A
1,1,10523666,G,A
2,1,11024271,G,T
3,1,15708540,A,G
4,1,17296361,C,G
...,...,...,...,...
535,22,41605809,G,C
536,22,42076330,G,A
537,22,43459864,G,A
538,22,45828202,C,T


In [21]:
if df1_pred.columns.tolist() != df2_ans.columns.tolist():
    df1_pred.columns = df2_ans.columns.tolist()

In [26]:
df1_pred['Chrom'] = df1_pred['Chrom'].astype(int)
df1_pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Chrom,Position,Ref,Alt
0,1,3745834,A,G
1,1,3745836,T,C
2,1,3745846,A,G
3,1,3745848,T,C
4,1,3745849,T,C
...,...,...,...,...
9823,22,50897995,G,A
9824,22,50898007,C,T
9825,22,50898026,G,A
9826,22,50898042,C,T


In [27]:
df2_ans['Chrom'] = df2_ans['Chrom'].astype(int)
df2_ans

,Chrom,Position,Ref,Alt
0,1,3745924,G,A
1,1,10523666,G,A
2,1,11024271,G,T
3,1,15708540,A,G
4,1,17296361,C,G
...,...,...,...,...
535,22,41605809,G,C
536,22,42076330,G,A
537,22,43459864,G,A
538,22,45828202,C,T


In [28]:
merge_diff = df2_ans.merge(df1_pred, indicator=True, how='outer')

In [29]:
merge_diff

,Chrom,Position,Ref,Alt,_merge
0,1,3745924,G,A,both
1,1,10523666,G,A,both
2,1,11024271,G,T,both
3,1,15708540,A,G,both
4,1,17296361,C,G,both
...,...,...,...,...,...
9833,22,50897993,G,A,right_only
9834,22,50897995,G,A,right_only
9835,22,50898007,C,T,right_only
9836,22,50898042,C,T,right_only


In [30]:
d = pd.DataFrame(merge_diff['_merge'].value_counts())

TP =int(d.loc['both'])
FN = int(d.loc["left_only"])
FP = int(d.loc["right_only"])

print(f'TP: {TP}')
print(f'FN: {FN}')
print(f'FP: {FP}')

TP = float(TP)
pred_n = float(len(df1_pred))
ans_n = float(len(df2_ans))

PPV = '{:.5f}'.format(TP / pred_n)
TPR ='{:.5f}'.format(TP / ans_n)

print(f'PPV(Precision): {PPV}')
print(f'TPR(Recall): {TPR}')

TP: 530
FN: 10
FP: 9298
PPV(Precision): 0.05393
TPR(Recall): 0.98148


In [45]:
TP = int(TP)
FN = int(FN)
FP = int(FP)

result_df = pd.DataFrame({'value': [f'{TP}', f'{FN}', f'{FP}', f'{PPV}', f'{TPR}']}, index=['TP', 'FN', 'FP', 'PPV(Precision)', 'TPR(Recall)'])
result_df

,value
TP,530
FN,10
FP,9298
PPV(Precision),0.05393
TPR(Recall),0.98148


In [47]:
result_df.to_csv('result.txt', sep = '\t', header=None)